<a href="https://colab.research.google.com/github/lucas-azdias/Artistic-Text-Recognition/blob/main/artefatos/modelos/MAERec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importações

In [1]:
import pathlib
import json
import sys

In [2]:
!git clone https://github.com/Mountchicken/Union14M.git maerec > /dev/null 2>&1
!git clone https://github.com/lucas-azdias/WordArt-V1.5-Dataset.git data/full_images > /dev/null 2>&1

In [3]:
!mv maerec/* . > /dev/null 2>&1
!rm -rf maerec > /dev/null 2>&1

In [10]:
!mv data/full_images/*/*.json* data > /dev/null 2>&1
!mkdir -p data/benchmark
!mv data/testB.json* data/benchmark > /dev/null 2>&1

In [ ]:
!wget -q -O Miniconda.sh https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda.sh -b -f -p miniconda > /dev/null
!rm Miniconda.sh

!sudo rm -f /usr/local/bin/conda
!ln -s /content/miniconda/bin/conda /usr/local/bin/conda

!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r

accepted Terms of Service for https://repo.anaconda.com/pkgs/main
accepted Terms of Service for https://repo.anaconda.com/pkgs/r


In [ ]:
!conda env remove -n maerec -y -q > /dev/null 2>&1
!conda create -n maerec python=3.8 -y -q > /dev/null 2>&1

!conda run -n maerec pip install -qq torch==1.8.1+cu111 torchvision==0.9.1+cu111 -f https://download.pytorch.org/whl/torch_stable.html > /dev/null 2>&1
!conda run -n maerec pip install -qq -U openmim matplotlib_inline ipython > /dev/null 2>&1
!conda run -n maerec mim install -qq mmengine mmcv==2.0.0 mmdet==3.0.0 > /dev/null 2>&1
!conda run -n maerec pip install -qq -r "mmocr-dev-1.x/requirements/albu.txt" > /dev/null 2>&1
!conda run -n maerec pip install -qq -r "mmocr-dev-1.x/requirements.txt" > /dev/null 2>&1
!conda run -n maerec pip install -qq -v -e "mmocr-dev-1.x/." --config-settings editable_mode=compat > /dev/null 2>&1
!conda run -n maerec pip install -qq timm==0.9.2 albumentations==1.3.1 > /dev/null 2>&1

In [ ]:
# !gdown -q 1OSGZ7Jt7q4Pj3M9cUQznJpRDSDjzBVuy -O mmocr-dev-1.x/vit.pth
!gdown -q 13E0cmvksKwvjNuR62xZhwkg8eQJfb_Hp -O mmocr-dev-1.x/maerec.pth

## Arquivos alterados

In [11]:
target_file = pathlib.Path(".") / "mmocr-dev-1.x" / "configs" / "textrecog" / "_base_" / "datasets" / "icdar2011.py"

new_text = """icdar2011_textrecog_data_root = '../data/'

icdar2011_textrecog_train = dict(
    type='OCRDataset',
    data_root=icdar2011_textrecog_data_root,
    ann_file='train.json',
    test_mode=False,
    pipeline=None)

icdar2011_textrecog_val = dict(
    type='OCRDataset',
    data_root=icdar2011_textrecog_data_root,
    ann_file='testA.json',
    test_mode=False,
    pipeline=None)
"""

target_file.write_text(new_text, encoding="utf-8")

381

In [12]:
target_file = pathlib.Path(".") / "mmocr-dev-1.x" / "configs" / "textrecog" / "_base_" / "datasets" / "union14m_benchmark.py"

new_text = """union14m_root = '../data'
union14m_benchmark_root = '../data/benchmark'

union14m_benchmark_general = dict(
    type='OCRDataset',
    data_prefix=dict(img_path=f'{union14m_root}/'),
    ann_file=f'{union14m_benchmark_root}/testB.json',
    test_mode=True,
    pipeline=None)
"""

target_file.write_text(new_text, encoding="utf-8")

276

In [13]:
target_file = pathlib.Path(".") / "mmocr-dev-1.x" / "configs" / "textrecog" / "_base_" / "datasets" / "union14m_train.py"

new_text = """union14m_data_root = '../data'

union14m_train = dict(
    type='OCRDataset',
    data_root=union14m_data_root,
    ann_file='train.json',
    test_mode=True,
    pipeline=None)

union14m_val = dict(
    type='OCRDataset',
    data_root=union14m_data_root,
    ann_file='testA.json',
    pipeline=None)
"""

target_file.write_text(new_text, encoding="utf-8")

303

In [14]:
target_file = pathlib.Path(".") / "mmocr-dev-1.x" / "configs" / "textrecog" / "_base_" / "schedules" / "schedule_adamw_cos_10e.py"

new_text = """# optimizer
optim_wrapper = dict(
    type='OptimWrapper',
    optimizer=dict(
        type='AdamW',
        lr=1e-5,
        betas=(0.9, 0.999),
        eps=1e-08,
        weight_decay=1e-4))
train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=5, val_interval=1)
val_cfg = dict(type='ValLoop')
test_cfg = dict(type='TestLoop')

# learning policy
param_scheduler = [
    dict(
        type='CosineAnnealingLR',
        T_max=10,
        eta_min=1e-6,
        convert_to_iter_based=True)
]
"""

target_file.write_text(new_text, encoding="utf-8")

493

In [15]:
target_file = pathlib.Path(".") / "mmocr-dev-1.x" / "configs" / "textrecog" / "maerec" / "_base_marec_vit_s.py"

new_text = """dictionary = dict(
    type='Dictionary',
    dict_file=  # noqa
    '{{ fileDirname }}/../../../dicts/english_digits_symbols_space.txt',
    with_padding=True,
    with_unknown=True,
    same_start_end=True,
    with_start=True,
    with_end=True)

model = dict(
    type='MAERec',
    backbone=dict(
        type='VisionTransformer',
        img_size=(32, 128),
        patch_size=(4, 4),
        embed_dim=384,
        depth=12,
        num_heads=6,
        mlp_ratio=4.0,
        qkv_bias=True,
        pretrained="vit.pth"),
    decoder=dict(
        type='MAERecDecoder',
        n_layers=6,
        d_embedding=384,
        n_head=8,
        d_model=384,
        d_inner=384 * 4,
        d_k=48,
        d_v=48,
        postprocessor=dict(type='AttentionPostprocessor'),
        module_loss=dict(
            type='CEModuleLoss', reduction='mean', ignore_first_char=True),
        max_seq_len=48,
        dictionary=dictionary),
    data_preprocessor=dict(
        type='TextRecogDataPreprocessor',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375]),
        )

train_pipeline = [
    dict(type='LoadImageFromFile', ignore_empty=True, min_size=0),
    dict(type='LoadOCRAnnotations', with_text=True),
    dict(type='Resize', scale=(128, 32)),
    dict(
        type='RandomApply',
        prob=0.5,
        transforms=[
            dict(
                type='RandomChoice',
                transforms=[
                    dict(
                        type='RandomRotate',
                        max_angle=15,
                    ),
                    dict(
                        type='TorchVisionWrapper',
                        op='RandomAffine',
                        degrees=15,
                        translate=(0.3, 0.3),
                        scale=(0.5, 2.),
                        shear=(-45, 45),
                    ),
                    dict(
                        type='TorchVisionWrapper',
                        op='RandomPerspective',
                        distortion_scale=0.5,
                        p=1,
                    ),
                ])
        ],
    ),
    dict(
        type='RandomApply',
        prob=0.25,
        transforms=[
            dict(type='PyramidRescale'),
            dict(
                type='mmdet.Albu',
                transforms=[
                    dict(type='GaussNoise', var_limit=(20, 20), p=0.5),
                    dict(type='MotionBlur', blur_limit=7, p=0.5),
                ]),
        ]),
    dict(
        type='RandomApply',
        prob=0.25,
        transforms=[
            dict(
                type='TorchVisionWrapper',
                op='ColorJitter',
                brightness=0.5,
                saturation=0.5,
                contrast=0.5,
                hue=0.1),
        ]),
    dict(
        type='PackTextRecogInputs',
        meta_keys=('img_path', 'ori_shape', 'img_shape', 'valid_ratio'))
]

test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', scale=(128, 32)),
    # add loading annotation after ``Resize`` because ground truth
    # does not need to do resize data transform
    dict(type='LoadOCRAnnotations', with_text=True),
    dict(
        type='PackTextRecogInputs',
        meta_keys=('img_path', 'ori_shape', 'img_shape', 'valid_ratio'))
]

tta_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='TestTimeAug',
        transforms=[
            [
                dict(
                    type='ConditionApply',
                    true_transforms=[
                        dict(
                            type='ImgAugWrapper',
                            args=[dict(cls='Rot90', k=0, keep_size=False)])
                    ],
                    condition="results['img_shape'][1]<results['img_shape'][0]"
                ),
                dict(
                    type='ConditionApply',
                    true_transforms=[
                        dict(
                            type='ImgAugWrapper',
                            args=[dict(cls='Rot90', k=1, keep_size=False)])
                    ],
                    condition="results['img_shape'][1]<results['img_shape'][0]"
                ),
                dict(
                    type='ConditionApply',
                    true_transforms=[
                        dict(
                            type='ImgAugWrapper',
                            args=[dict(cls='Rot90', k=3, keep_size=False)])
                    ],
                    condition="results['img_shape'][1]<results['img_shape'][0]"
                ),
            ],
            [dict(type='Resize', scale=(128, 32))],
            # add loading annotation after ``Resize`` because ground truth
            # does not need to do resize data transform
            [dict(type='LoadOCRAnnotations', with_text=True)],
            [
                dict(
                    type='PackTextRecogInputs',
                    meta_keys=('img_path', 'ori_shape', 'img_shape',
                               'valid_ratio'))
            ]
        ])
]"""

target_file.write_text(new_text, encoding="utf-8")

5076

In [16]:
target_file = pathlib.Path(".") / "mmocr-dev-1.x" / "configs" / "textrecog" / "maerec" / "maerec_b_union14m.py"

new_text = """# training schedule for 1x
_base_ = [
    '_base_marec_vit_s.py',
    # '../_base_/datasets/union14m_train.py',
    '../_base_/datasets/union14m_benchmark.py',
    # '../_base_/datasets/cute80.py',
    # '../_base_/datasets/iiit5k.py',
    # '../_base_/datasets/svt.py',
    # '../_base_/datasets/svtp.py',
    '../_base_/datasets/icdar2011.py',
    # '../_base_/datasets/icdar2015.py',
    '../_base_/default_runtime.py',
    '../_base_/schedules/schedule_adamw_cos_10e.py',
]

# load_from = 'https://download.openmmlab.com/mmocr/textrecog/mae/mae_union14m/maerec_b_union14m-4b98d1b4.pth'
load_from = "./maerec.pth"

model = dict(
    backbone=dict(
        type='VisionTransformer',
        img_size=(32, 128),
        patch_size=4,
        embed_dim=768,
        depth=12,
        num_heads=12,
        mlp_ratio=4.0,
        qkv_bias=True,
        pretrained=None),
    decoder=dict(
        type='MAERecDecoder',
        n_layers=6,
        d_embedding=768,
        n_head=8,
        d_model=768,
        d_inner=3072,
        d_k=96,
        d_v=96))

# dataset settings
train_list = [
    _base_.icdar2011_textrecog_train
]
val_list = [
    _base_.icdar2011_textrecog_val
]
test_list = [
    _base_.union14m_benchmark_general,
]

default_hooks = dict(logger=dict(type='LoggerHook', interval=50))

auto_scale_lr = dict(base_batch_size=128)

train_dataset = dict(
    type='ConcatDataset', datasets=train_list, pipeline=_base_.train_pipeline)
test_dataset = dict(
    type='ConcatDataset', datasets=test_list, pipeline=_base_.test_pipeline)
val_dataset = dict(
    type='ConcatDataset', datasets=val_list, pipeline=_base_.test_pipeline)

train_dataloader = dict(
    batch_size=64,
    num_workers=1,
    persistent_workers=True,
    pin_memory=True,
    sampler=dict(type='DefaultSampler', shuffle=True),
    dataset=train_dataset)

test_dataloader = dict(
    batch_size=128,
    num_workers=1,
    persistent_workers=True,
    pin_memory=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=test_dataset)

val_dataloader = dict(
    batch_size=128,
    num_workers=1,
    persistent_workers=True,
    pin_memory=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=val_dataset)

val_evaluator = dict(
    dataset_prefixes=['testA'])


test_evaluator = dict(dataset_prefixes=[
    'general'
])
"""

target_file.write_text(new_text, encoding="utf-8")

2385

# Treinamento

In [ ]:
!cd mmocr-dev-1.x && conda run -n maerec python tools/train.py configs/textrecog/maerec/maerec_b_union14m.py --amp

10/25 18:14:26 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.20 (default, Oct  3 2024, 15:24:27) [GCC 11.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 990883938
    GPU 0: NVIDIA L4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 12.5, V12.5.82
    GCC: gcc (Ubuntu 11.4.0-1ubuntu1~22.04.2) 11.4.0
    PyTorch: 1.8.1+cu111
    PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v1.7.0 (Git Hash 7aed236906b1f7a05c0917e5257a1af05e9ff683)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.1
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,co

In [ ]:
# URL do repositório
maerec_url = "https://github.com/Mountchicken/Union14M.git"
maerec_path = pathlib.Path("/content/maerec")

# Clonar o repositório
!rm -rf "{maerec_path}"
!git clone "{maerec_url}" "{maerec_path}"

if not str(maerec_path.absolute()) in sys.path:
    sys.path.append(str(maerec_path.absolute()))

Cloning into '/content/maerec'...
remote: Enumerating objects: 2963, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 2963 (delta 7), reused 0 (delta 0), pack-reused 2942 (from 1)
Receiving objects: 100% (2963/2963), 22.58 MiB | 15.85 MiB/s, done.
Resolving deltas: 100% (990/990), done.


In [ ]:
def load_data(base_path: pathlib.Path, name: str) -> tuple[list[pathlib.Path], list[str]]:
    labels_path = pathlib.Path(base_path, name, "labels.txt")

    image_paths = []
    classes = []

    with open(labels_path, "r") as f:
        for line in f:
            # Ex: "train_image\320.png Ford"
            splitted = line.strip().split(" ")
            path, label = splitted[0].replace("\\", "/"), " ".join(splitted[1:])

            # Ajustar o caminho para apontar para a pasta correta
            img_path = pathlib.Path(base_path, name, pathlib.Path(path))
            image_paths.append(img_path)
            classes.append(label)

    return image_paths, classes

testB_images, testB_labels = load_data(pathlib.Path("data/full_images"), "testB")

In [ ]:
for filename in pathlib.Path("/content/mmocr-dev-1.x/work_dirs/maerec_b_union14m/").rglob("*.pth"):
    infer_imgs = f"data/full_images/testB/images"

    !rm -rf "./output"
    !mkdir -p "./output"

    !cd "maerec/mmocr-dev-1.x" && conda run -n maerec python "tools/infer.py" \
        "../../{infer_imgs}" \
        --batch-size 256 \
        --out-dir "../../output" \
        --rec "configs/textrecog/maerec/maerec_b_union14m.py" \
        --rec-weights "{filename}" \
        --device cuda \
        --show \
        --save_pred \
        > /dev/null 2>&1

    results = []
    for json_file in pathlib.Path("./output").rglob("*.json"):
        with open(json_file, "r", encoding="utf-8") as file:
            data = json.load(file)
        results.append((json_file, data["rec_texts"][0], data["rec_scores"][0]))

    correct = 0
    for img_path, label in zip(testB_images, testB_labels):
        for path, pred, _ in results:
            if pathlib.Path(img_path).stem == pathlib.Path(path).stem:
                correct += 1 if pred == label else 0
                break

    print(f"{filename}: {correct}/{len(results)} ({correct / len(results) * 100:.2f}%)")

Loads checkpoint by local backend from path: /content/mmocr-dev-1.x/work_dirs/maerec_b_union14m/epoch_3.pth
10/25 18:31:46 - mmengine - WARNING - Failed to search registry with scope "mmocr" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmocr" is a correct scope, or whether the registry is initialized.












Inference ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1 it/s  

/content/miniconda/envs/maerec/lib/python3.8/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '

/content/mmocr-dev-1.x/work_dirs/maerec_b_union14m/epoch_3.pth: 2439/3000 (81.30%)
Loads checkpoint by local backend from path: /content/mmocr-dev-1.x/work_dirs/maere